In [9]:
!pip install ultralytics

from IPython import display
display.clear_output()
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import moviepy.video.fx.all as vfx
import pandas as pd
from PIL import Image
import csv
import math
from ultralytics  import YOLO
import time
from keras.models import load_model
from PIL import Image
import csv

In [10]:
def divide_image1(image, output_directory, rows, cols):
    # Ouvrir l'image
    img = Image.open(image_path)

    # Obtenir les dimensions de l'image
    img_width, img_height = img.size

    # Calculer les dimensions d'une zone
    zone_width = img_width // cols
    zone_height = img_height // rows

    # Diviser l'image en zones
    with open(f"{output_directory}/zones_info.csv", "w", newline="") as csvfile:
        fieldnames = ["zone_id", "x0", "y0", "x1", "y1"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in range(rows):
            for col in range(cols):
                # Coordonnées du coin supérieur gauche de la zone
                x0 = col * zone_width
                y0 = row * zone_height

                # Coordonnées du coin inférieur droit de la zone
                x1 = x0 + zone_width
                y1 = y0 + zone_height

                # Extraire la zone de l'image
                zone_img = img.crop((x0, y0, x1, y1))

                # Sauvegarder la zone dans un fichier
                zone_img.save(f"{output_directory}/zone_{row}_{col}.png")

                # Écrire les informations de la zone dans le fichier CSV
                writer.writerow({"zone_id": f"zone_{row}_{col}", "x0": x0, "y0": y0, "x1": x1, "y1": y1})

# Appeler la fonction avec le chemin de votre image et le répertoire de sortie
image_path = "/content/Swim-safety_blog-banner.jpg"
output_directory = "/content/"
rows = 3
cols = 3
divide_image1(image_path, output_directory, rows, cols)

In [11]:
# YOLO class definition is assumed to be defined elsewhere in the code.

def detect_objects_in_zone(zone_image_path, model_path, conf_val, output_directory, iou, agnostic_nms):
    # Create an instance of the YOLO model using the specified model path
    model = YOLO(model_path)

    # Read the input image and convert it to a NumPy array in BGR format
    frame = Image.open(zone_image_path)
    frame_np = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

    # Perform object detection using the YOLO model on the input image
    result = model.predict(frame, conf=conf_val, classes = 0, iou = iou,agnostic_nms = agnostic_nms, show=False, verbose=False)

    # Count the number of swimmers detected
    boxes = result[0].boxes
    num_swimmers = 0
    for box in boxes:
        confidence = box.conf[0]
        confidence = math.ceil(confidence * 100)
        if confidence > 25:
            num_swimmers += 1
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # Draw a rectangle around the detected object on the image
            cv2.rectangle(frame_np, (x1, y1), (x2, y2), (0, 0, 255), 5)

    # Draw text on the image to display the number of swimmers detected
    cv2.putText(frame_np, f'{"num_swimmers :"} {num_swimmers}', (5, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 4)

    # Save the modified image with rectangles and text
    filename = os.path.basename(zone_image_path)
    filename = os.path.splitext(filename)[0]  # Remove the file extension
    new_filename = f"{filename}_{num_swimmers}.png"
    output_path = os.path.join(output_directory, new_filename)
    cv2.imwrite(output_path, frame_np)

    # Return the total number of swimmers detected
    return num_swimmers

# Example usage:
row = 2
col = 2
zone_image_path = f"/content/zone_{row}_{col}.png"
model_path = 'yolov8x.pt'
conf_val = 0.28
iou_vl = 0.4
agnostic_nms_vl = True
output_directory = "/content/"
num_swimmers = detect_objects_in_zone(zone_image_path, model_path, conf_val, output_directory, iou = iou_vl,agnostic_nms = agnostic_nms_vl)

100%|██████████| 131M/131M [00:00<00:00, 220MB/s]


In [12]:
model_path = 'yolov8x.pt'
output_directory = "/content/"
list_num_swimmers = []
for row in range(rows):
  for col in range(cols):
    zone_image_path = f"/content/zone_{row}_{col}.png"
    num_swimmers = detect_objects_in_zone(zone_image_path, model_path, conf_val, output_directory, iou = iou_vl,agnostic_nms = agnostic_nms_vl)
    list_num_swimmers.append(num_swimmers)

In [13]:
# Read the existing CSV file into a pandas DataFrame
csv_file_path = "/content/zones_info.csv"
df_zones = pd.read_csv(csv_file_path)

# Add the 'num_swimmers' column to the DataFrame
df_zones['num_swimmers'] = list_num_swimmers

# Save the updated DataFrame back to the CSV file
df_zones.to_csv(csv_file_path, index=False)

In [14]:
zones = rows * cols
zone_levl = 0
zones_a_masque = []
for zone_id in range(zones):
  zone_levl += 1
  num_swimmers = df_zones.loc[zone_id, 'num_swimmers']
  if (zone_levl < 4 and num_swimmers > 8):
    zones_a_masque.append(zone_id)
  if (zone_levl >= 4 and zone_levl < 7 and num_swimmers > 5):
    zones_a_masque.append(zone_id)
  if (zone_levl >= 7 and zone_levl < 10 and num_swimmers > 2):
    zones_a_masque.append(zone_id)

zones_a_masque

[4, 5, 6, 7, 8]

In [15]:
# Charger l'image
image = cv2.imread("/content/Swim-safety_blog-banner.jpg")

# Créer un masque vide (tout en blanc)
mask = np.ones(image.shape, dtype=np.uint8) * 255

# Boucle pour masquer chaque zone
for zone_id in zones_a_masque:
    x0 = df_zones.loc[zone_id, 'x0']
    y0 = df_zones.loc[zone_id, 'y0']
    x1 = df_zones.loc[zone_id, 'x1']
    y1 = df_zones.loc[zone_id, 'y1']

    # Créer un masque pour la zone à masquer (tout en noir)
    cv2.rectangle(mask, (x0, y0), (x1, y1), (0, 0, 0), -1)

# Masquer les zones dans l'image
masked_image = cv2.bitwise_and(image, mask)

# Afficher l'image masquée
cv2.imwrite("/content/masked_image.png", masked_image)


True